<a href="https://colab.research.google.com/github/LM1997610/Networking-for-Big-Data/blob/main/NBD_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBD - Challenge 2

In [2]:
import time; import datetime

import numpy as np
import pandas as pd
import random as rdm
from tqdm.auto import tqdm

from tabulate import tabulate
from matplotlib import pyplot as plt

import math

In [3]:
!gdown 19baCZ9_wVNSBWVRJFtDIA4haBxv6IXAw

Downloading...
From: https://drive.google.com/uc?id=19baCZ9_wVNSBWVRJFtDIA4haBxv6IXAw
To: /content/Cell_a.zip
100% 43.3M/43.3M [00:00<00:00, 166MB/s]


### Data loading

In [4]:
data = pd.read_csv('Cell_a.zip')

In [5]:
print("\n data.shape : {}\n".format((data.shape))) #2329133
data.head()


 data.shape : (2329133, 5)



,Job_ID,Task_ID,Arrival_Time,CPU,Memory
0,375000667413,0,603026300,0.041851,0.001169
1,375000669289,0,606413041,0.024968,0.001179
2,375000670586,0,608994453,0.024176,0.001173
3,375000670588,0,608994466,0.019552,0.001163
4,375000670590,0,609042903,0.028044,0.001179


### Constants

In [6]:
N = 64
mu = 0.1

# Explorative data analysis

In [ ]:
def eCCDF(job:int, field:str, d=data) -> np.array:
    sub = d[d['Job_ID']==job]
    arr = np.array(sub[field])
    quantiles, counts = np.unique(arr, return_counts=True)
    cumprob = np.cumsum(counts).astype(np.double)/arr.size
    return quantiles, 1-cumprob

In [ ]:
count = data.groupby('Job_ID')["Task_ID"].count()
mean_cpu = data.groupby('Job_ID')["CPU"].mean()
mean_memory = data.groupby('Job_ID')["Memory"].mean()
var_cpu = data.groupby('Job_ID')["CPU"].var()
var_memory = data.groupby('Job_ID')["Memory"].var()

In [ ]:
jobs = pd.unique(data['Job_ID'])
temp = {job:k for (job,k) in zip(jobs, count)}

multitask = [job for job in jobs if temp[job]>1]

eCCDF_CPU = {job:0 for job in jobs}
for job in tqdm(multitask):
    eCCDF_CPU[job] = list(zip(eCCDF(job,'CPU')))

eCCDF_memory = {job:0 for job in jobs}
for job in tqdm(multitask):
    eCCDF_memory[job] = list(zip(eCCDF(job,'Memory')))

100%|██████████| 70702/70702 [06:12<00:00, 189.93it/s]


In [ ]:
stats = pd.DataFrame({
    'Jobs':jobs,
    'Tasks_count':count,
    'Mean_CPU':mean_cpu,
    'Mean_Memory':mean_memory,
    'CPU_variance':var_cpu,
    'Memory_variance':var_memory,
    'CPU_eCCDF': [list(value) if value != 0 else value for value in eCCDF_CPU.values()],
    'Memory_eCCDF': [list(value) if value != 0 else value for value in eCCDF_memory.values()]
})

In [ ]:
print("\n stats.shape : {}\n".format((stats.shape)))

stats.head()


 stats.shape : (1163061, 8)



,Jobs,Tasks count,Mean CPU,Mean Memory,CPU variance,Memory variance,CPU eCCDF,Memory eCCDF
Job_ID,,,,,,,,
375000667413,375000667413,1,0.041851,0.001169,NaN,NaN,0,0
375000669289,375000669289,1,0.024968,0.001179,NaN,NaN,0,0
375000670586,375000670586,1,0.024176,0.001173,NaN,NaN,0,0
375000670588,375000670588,1,0.019552,0.001163,NaN,NaN,0,0
375000670590,375000670590,1,0.028044,0.001179,NaN,NaN,0,0


# Implementing the baselines: LWL for dispatching, FCFS for scheduling

## LWL policy for dispatching

## FCFS for scheduling

In [1]:
!git clone https://github.com/LM1997610/Networking-for-Big-Data NBD

Cloning into 'NBD'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 29 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), 7.96 KiB | 1019.00 KiB/s, done.


In [7]:
from NBD.module import FCFS

In [8]:
# fake queue for testing purposes

server_dict = [data.iloc[i].to_dict() for i in range(20)]

In [9]:
def highlight_max(cell):

    if type(cell) != str and cell < 0 :

        return 'color: red'
    else:
        return 'color: green'

In [10]:
FCFS.Scheduling(server_dict).style.applymap(highlight_max, subset=pd.IndexSlice[:, ["execution_time", 'waiting']])

# negative waiting_time --> no queue for this job and server stays jobless for a while 


 Mean Waiting Time: -1089372.4166503907 



,Job_id,Time_deltas,execution_time,exit_time,turn_around,waiting
0,375000667413,0,418.510437,418.510437,418.510437,0.000000
1,375000669289,3386741,249.681473,668.191910,-3386072.808090,-3386322.489563
2,375000670586,2581412,241.756439,909.948349,-2580502.051651,-2580743.808090
3,375000670588,13,195.522308,1105.470657,1092.470657,896.948349
4,375000670590,48437,280.437469,1385.908127,-47051.091873,-47331.529343
5,375000670901,751754,279.102325,1665.010452,-750088.989548,-750368.091873
6,375000670926,28007,303.726196,1968.736649,-26038.263351,-26341.989548
7,375000670981,412680,184.736252,2153.472900,-410526.527100,-410711.263351
8,375000671303,747053,250.272751,2403.745651,-744649.254349,-744899.527100
9,375000671907,4614372,87446.479797,89850.225449,-4524521.774551,-4611968.254349


_______________________________________________________

In [14]:
######################################################### 

def random_dispatcher(dizionario):  #diz da assegnare alle coda

    server_selected = rdm.randrange(0, N)
    
    if servers_queues[server_selected] == None:
      servers_queues[server_selected] = [dizionario]
    else:
      servers_queues[server_selected].append(dizionario)
    

In [15]:
servers_queues =  {x:None for x in range(0,N)}
#servers_queues

In [16]:
st = time.time()

for i in tqdm(range(len(data))):

    diz = data.iloc[i].to_dict()

    random_dispatcher(diz)

et = time.time()
print('\n Execution time:', str(datetime.timedelta(seconds = et - st))[:10] , 'mins')

  0%|          | 0/2329133 [00:00<?, ?it/s]


 Execution time: 0:02:44.85 mins


In [60]:
print("Length each q:", [len(x) for x in servers_queues.values()])

sum([len(x) for x in servers_queues.values()]) == data.shape[0]

Length each q: [36331, 36404, 36327, 36416, 36473, 36284, 36369, 36394, 36656, 36196, 36384, 36208, 36333, 36348, 36596, 36275, 36329, 36386, 36442, 36258, 36257, 36282, 36424, 36094, 36734, 36430, 36481, 36539, 36597, 36508, 36386, 36359, 36604, 36660, 36180, 36199, 36430, 36184, 36368, 36559, 36374, 36139, 36449, 36383, 36584, 36522, 36331, 36429, 36543, 36505, 36558, 36529, 36157, 36288, 36261, 36560, 36248, 36486, 36199, 36159, 36446, 36212, 36836, 36251]


True

In [31]:
list_of_df = []

for key in tqdm(servers_queues.keys()):
  x = FCFS.Scheduling(servers_queues[key])
  list_of_df.append(x)

  0%|          | 0/64 [00:00<?, ?it/s]


 Mean Waiting Time: 415167100.3461585 


 Mean Waiting Time: 250064282.96249714 


 Mean Waiting Time: 286693987.2899582 


 Mean Waiting Time: 413098050.0932615 


 Mean Waiting Time: 337398195.7342776 


 Mean Waiting Time: 427849264.01930887 


 Mean Waiting Time: 285267200.8763812 


 Mean Waiting Time: 426544390.73153186 


 Mean Waiting Time: 396517227.7389145 


 Mean Waiting Time: 336952442.39545983 


 Mean Waiting Time: 481480902.5859111 


 Mean Waiting Time: 388391328.93934137 


 Mean Waiting Time: 278226283.66535467 


 Mean Waiting Time: 435070739.9263109 


 Mean Waiting Time: 236079383.5705526 


 Mean Waiting Time: 449796118.15531266 


 Mean Waiting Time: 456189232.7325468 


 Mean Waiting Time: 348784682.6409999 


 Mean Waiting Time: 316789498.3767077 


 Mean Waiting Time: 446168863.2933486 


 Mean Waiting Time: 544360836.1222407 


 Mean Waiting Time: 432044584.1599811 


 Mean Waiting Time: 412386909.1750247 


 Mean Waiting Time: 443408416.683588 


 Mean Wai

In [ ]:
## TO DO:
# re-group by Job_ID and evaluate Job response time
# then Job slowdown, Utilization coefficient of server,  Messaging load (?) 

In [59]:
df = pd.concat(list_of_df)
df = df.sort_values(by="Job_id")
df.head()

,Job_id,Time_deltas,execution_time,exit_time,turn_around,waiting
0,375000667413,0,418.510437,418.510437,418.510437,0.0
0,375000669289,0,249.681473,249.681473,249.681473,0.0
0,375000670586,0,241.756439,241.756439,241.756439,0.0
0,375000670588,0,195.522308,195.522308,195.522308,0.0
0,375000670590,0,280.437469,280.437469,280.437469,0.0


In [ ]:
# JOB_RESPONSE_TIME = execution_time + waiting_time for all its task 

# ma se due task dello stesso job sono svolte in contemporanea su due server diversi?
# c'è da riprendere il tempo assoluto di arrivo 